# Introduction

Welcome to the seminar! We hope you will find it useful and fun.
Before we start, let's make sure you have everything ready.

## Installing python

If you are reading this inside a jupyter notebook, likely it's already installed.
Besides python, you will need `jupyter`, `numpy` and `julia` libraries, you can install them using `pip install`.
We have tested this notebook under python3, but python2 should work too.

## Installing julia

You will also need to install the julia interpreter. Go to https://julialang.org/downloads/ and get yourself a v1.1+ that suits your operating system.
If you download julia as a binary, unpack it somewhere and add this to your `.bashrc`:
```
export PATH="/path/to/julia/bin:$PATH
```
You might need to reload kernel after you update `PATH`.

Type `julia` in the terminal, if you see a nice ascii art with a julia logo and the interpreter prompt it worked well. Now you need to install the AdaGram package (lives here: https://bitbucket.org/sbos/adagram_deepbayes2019). In the julia interpreter type in the following command:
```
using Pkg
Pkg.add(PackageSpec(url="https://bitbucket.org/sbos/adagram_deepbayes2019.git"))
```

## Download an AdaGram model

If you have wget installed, you can simply run the cell below.

In [2]:
!wget https://w2v.s3.amazonaws.com/huang_super_200D_0.1_min20_hs_t1e-17.model

/bin/sh: wget: command not found


Otherwise, download this file manually to the same dir where this notebook is located (or remember the path and modify it below accordingly)

Also, `git clone https://bitbucket.org/sbos/adagram_deepbayes2019.git` somewhere on your laptop. Further we will refer to this directory as `./adagram_deepbayes2019`.

## Check that everything works

In [3]:
from julia.api import Julia
# if this cell fails, uncomment the line below
# jl = Julia(compiled_modules=False)
import julia
julia.install()
from julia import AdaGram

In [4]:
vm, vocab = AdaGram.load_model("huang_super_200D_0.1_min20_hs_t1e-17.model")

### Prior probabilities of senses of the word "apple"

In [6]:
AdaGram.expected_pi(vm, vocab.word2id["apple"])

array([5.07910968e-01, 2.38101072e-01, 2.53984509e-01, 3.13751697e-06,
       2.85204898e-07, 2.59277179e-08, 2.35706526e-09, 2.14278660e-10,
       1.94798782e-11, 1.77089802e-12, 1.60990729e-13, 1.46355208e-14,
       1.33050189e-15, 1.20954717e-16, 1.09958834e-17, 9.99625763e-19,
       9.08750694e-20, 8.26136994e-21, 7.51033631e-22, 6.82757847e-23,
       6.20688951e-24, 5.64262683e-25, 5.12966076e-26, 4.66332796e-27,
       4.23938905e-28, 3.85399005e-29, 3.50362732e-30, 3.18511574e-31,
       2.89555977e-32, 2.89555977e-33])

### Nearest neighbours of the first sense

In [9]:
AdaGram.nearest_neighbors(vm, vocab, "apple", 1, min_count=2.)

[('macintosh', 1, 0.823969841003418),
 ('computers', 1, 0.7634434103965759),
 ('ibm', 1, 0.7630185484886169),
 ('intel-based', 1, 0.7424857020378113),
 ('iigs', 1, 0.7415772676467896),
 ('pc', 1, 0.7379290461540222),
 ('ms-dos', 1, 0.7352752685546875),
 ('kaypro', 1, 0.7326763272285461),
 ('powerpc-based', 1, 0.7302876114845276),
 ('dos', 1, 0.7282707095146179)]

### Nearest neighbours of the first sense

In [11]:
AdaGram.nearest_neighbors(vm, vocab, "apple", 2, min_count=2.)

[('pomegranate', 1, 0.8346080183982849),
 ('almond', 1, 0.8157211542129517),
 ('apricot', 1, 0.8051078915596008),
 ('plum', 1, 0.7945712804794312),
 ('peach', 1, 0.7862921357154846),
 ('cherry', 1, 0.7756718993186951),
 ('tamarind', 1, 0.7648524641990662),
 ('pear', 1, 0.7564710378646851),
 ('lemon', 1, 0.7523407936096191),
 ('blueberry', 1, 0.7521582245826721)]

### Disambiguation

In [12]:
AdaGram.disambiguate(vm, vocab, "apple", "fresh tasty breakfast".split(' '))

array([4.56448855e-05, 9.81135272e-01, 1.88190832e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00])

If that worked well, we are ready to start!

# Theory

## How many clusters one should expect in a Dirichlet process mixture model?

Remember from the lecture, that a measure $G \sim DP(\alpha, H)$ has a countable number of distinct values of $\theta \sim G$. This allows us to use Dirichlet processes as priors over assignments, e.g. cluster assingments in a mixture model.

Formally, if $\theta_1, \ldots, \theta_n \sim G$ one can write the following predictive distribution over $\theta_{n+1}$:

$$
    \theta_{n+1} | \theta_{1}, \ldots, \theta_{n} \sim \frac{1}{\alpha + n} ( \alpha H + \sum_{i=1}^n \delta(\theta - \theta_i)).
$$

From this formula, it is clear that some $\theta$s will be equal. Defining $K$ as the number of distinct $\theta$s and $n_k$ as the number of $\theta$s equal to the $k$-th value $\theta_k$, we can rewrite this equation:

$$
    \theta_{n+1} | \theta_{1}, \ldots, \theta_{n} \sim \frac{1}{\alpha + n} ( \alpha H + \sum_{k=1}^K n_k \delta(\theta - \theta_k)).
$$

This gives rise to the famous Chinese Restaurant Process, where each $\theta_k$ is a parameter associated with the $k$-th table. The $n+1$-th customer is choosing between the existing $K$ tables, each of which can be chooses with probability $\propto n_k$ and creating a new one with probability $\propto \alpha$.
If the $n+1$-th customer chooses $\theta_k$, then $n_k$ increases  by 1, otherwise $\theta_{K+1}$ gets sampled from the base measure $H$ and we set $n_{K+1} = 1$. This procedure then repeats.

Now, when we know this, let's try to think how many clusters one should get asymptotically as $n \rightarrow \infty$ in expectation? 

**Answer**: $\mathbb{E} K = $

## Variational approximation for the truncated stick-breaking process

Consider the standard stick-breaking construction of the Dirichlet process.
1. Stick-breaking proportions are sampled. $\beta_k \sim \text{Beta}(1, \alpha), \quad k=1,\ldots,\infty$.
2. Parameters are sampled. $\theta_k \sim H, \quad k=1,\ldots,\infty$. 
3. Objects are assigned to clusters according to the stick lengths $\pi_k$. $p(z_i = k) = \pi_k, \quad \pi_k = \beta_k \prod_{t=1}^{k-1} (1 - \beta_t)$
4. $x_i | z_i \sim p(\cdot | \theta_{z_i})$

After we observe data $\mathbf{x} = \{ x_1, x_2, \ldots, x_n \}$ we want to infer the posterior over DP parameters, in this case, $\mathbf{\beta}$ and $\boldsymbol{\theta}$ as well as the cluster assignments $\mathbf{z}$.
As it is often the case at this summer school, we wish to do that using variational inference. 

We choose a **finite**, fully-factorized family of variational approximations:
$$
    q(\mathbf{\beta}, \mathbf{z}, \boldsymbol{\theta}) = \prod_{k=1}^K \left[ q(\beta_k) q(\theta_k) \right] \prod_{i=1}^n q(z_i) \approx p(\mathbf{\beta}, \mathbf{z}, \boldsymbol{\theta} | \mathbf{x}).
$$
In this approximation, only $K$ clusters are modelled and we choose $q(\beta_K = 1) = 1$ which automatically assigns zero probability mass to all succeeding clusters.

Assume you are given with $q(z_i)$ for each object $x_i$. Derive a variational update for $q^*(\boldsymbol{\beta}) = \arg\min_{q(\boldsymbol{\theta})} \text{KL}( q(\mathbf{\beta}, \mathbf{z}, \boldsymbol{\theta}) || p(\mathbf{\beta}, \mathbf{z}, \boldsymbol{\theta} | \mathbf{x}))$.

**Answer**: $q^*(\beta_k) = $

**Bonus question**: look at the resulting parametrization. Does it look anyhow redundant? Could you use less parameters to fully describe $q^*(\boldsymbol{\beta})$?

# Practice

Remember you have just computed the expected number of clusters in a DP?
Let's see if the AdaGram model follows this analysis.

Plot the number of senses found for each word (the function `AdaGram.expected_pi` used a few cells above is helpful). Choose a reasonable treshold probability to prune out unused samples. 

In [24]:
# The set of all words known to the model
print("Total words: ", len(vocab.word2id.keys()))

# Word frequencies
print(vm.frequencies[vocab.word2id["apple"]])

Total words:  448927
28895


Now let's train some models! Unfortunately, many interesting properties of AdaGram can only be assessed when training on a relatively large corpus. It may be complicated during the time and compute limited practical session, so instead we will train models on synthetic data.

In our example, the word **a** can be encountered in two different contexts, one described by words **b** and **c** and the other one by **f** and **g**. So a good model should be able to discover these two "senses" of **a**.

In [30]:
# generating train data
for _ in range(100):
    !echo "b c b c b c a b c b c b b g f g a g g f g a a g f g" >> synth_train.txt

# generating test data
for _ in range(100):
    !echo "a c b b b a b c c f g g a g f g g a b c a b b c a c b f g f g a g f a g" >> synth_test.txt   
    
# you are more than welcome to generate something more interesting

In [31]:
!git clone https://bitbucket.org/sbos/adagram_deepbayes2019.git

Cloning into 'adagram_deepbayes2019'...
remote: Counting objects: 509, done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 509 (delta 333), reused 509 (delta 333)B/s     
Receiving objects: 100% (509/509), 10.13 MiB | 148.00 KiB/s, done.
Resolving deltas: 100% (333/333), done.


We will now prepare a dictionary file, this is needed only once

In [33]:
!./adagram_deepbayes2019/utils/dictionary.sh ./synth_train.txt ./synth.vocab

To train a model run this command:

In [34]:
!julia ./adagram_deepbayes2019/train.jl --window 3 --min-freq 1 --prototypes 3 \
       --alpha 1. --epochs 100 synth_train.txt synth.vocab synth_prot3_alpha1.model 

Building dictionary... Done!
      From worker 2:	64000 words read, 3412/5200
      From worker 2:	3.85% -1.3335 0.0240 0.0240 2.98/3.00 24.45 kwords/sec
      From worker 2:	7.69% -1.3210 0.0231 0.0231 2.99/3.00 40.15 kwords/sec
      From worker 2:	11.54% -1.3065 0.0221 0.0221 2.99/3.00 38.05 kwords/sec
      From worker 2:	15.38% -1.2951 0.0212 0.0212 3.00/3.00 39.41 kwords/sec
      From worker 2:	19.23% -1.2881 0.0202 0.0202 3.00/3.00 39.07 kwords/sec
      From worker 2:	23.08% -1.2833 0.0192 0.0192 3.00/3.00 38.95 kwords/sec
      From worker 2:	64000 words read, 1624/5200
      From worker 2:	28.46% -1.2785 0.0179 0.0179 3.00/3.00 39.66 kwords/sec
      From worker 2:	32.31% -1.2758 0.0169 0.0169 3.00/3.00 37.32 kwords/sec
      From worker 2:	36.15% -1.2737 0.0160 0.0160 3.00/3.00 36.16 kwords/sec
      From worker 2:	40.00% -1.2718 0.0150 0.0150 3.00/3.00 37.71 kwords/sec
      From worker 2:	43.85% -1.2702 0.0140 0.0140 3.00/3.00 37.34 kwords/sec
      From worker 2:	47.69% 

Pay attention to the parameters `--prototypes` and `--alpha`. Here we used allowed the DP to have up to 3 mixture component for each word and set $\alpha = 1$.

Now we can assess the test likelihood of our model.

In [36]:
!julia ./adagram_deepbayes2019/likelihood.jl --window 3 synth_prot3_alpha1.model synth_test.txt

      From worker 2:	3400 words read, 7200/7200
      From worker 2:	0 words read, 7200/7200
-1.6320696059620439
-1.6320696059620439


The last number printed is the likelihood we are looking for. 

Now play with the parameters and see if you can extract two different senses of the word **a**. 
Is this model better than the standard skip-gram (`--prototypes 1`) in terms of the test likelihood?
If you forgot how to use AdaGram, see examples in the introduction.